In [1]:
import pandas as pd
import re
from ast import literal_eval
from tqdm import tqdm
from ftfy import fix_text
# from util import UnitConversion, mapping_list_values, perl_to_posix
from groupby_toolz import enrich_db, gcloud
from flashtext import KeywordProcessor
from groupby_toolz import enrich_db, gcloud
pd.set_option('display.max_colwidth', None)
import ast
from fractions import Fraction
import warnings
warnings.filterwarnings('ignore')

In [8]:
# parameters
customer_id = '5'
formatted_attribute = 'Size Depth Cookware'
attributes = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Multiple Buckets"""

regex_pattern=r'(?i)(x\s?\d+.?\d*\".?D)|(.?.?.?.?.?Depth.?.?.?.?)()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attributes': attributes,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
curated_by=762

In [9]:
def query_from_file(file_name, params):
    with open(f'{file_name}', mode='r') as f:
        text = f.read()
        query = text.format(**params)
        return enrich_db(query)

In [13]:
customer_id = '5'
df = query_from_file(file_name='query/curated_by.sql', params=params)
df['date']=pd.to_datetime(df['date'])
df

attributes    snake_case_name     value  customer_id  \
0       Application Style  application_style  Brush-On            5   
1       Application Style  application_style  Brush-On            5   
2       Application Style  application_style  Brush-On            5   
3       Application Style  application_style  Brush-On            5   
4       Application Style  application_style  Brush-On            5   
...                   ...                ...       ...          ...   
402032    Weight Capacity    weight_capacity       n/a            5   
402033    Weight Capacity    weight_capacity       n/a            5   
402034    Weight Capacity    weight_capacity       n/a            5   
402035    Weight Capacity    weight_capacity       n/a            5   
402036    Weight Capacity    weight_capacity       n/a            5   

       external_id curation_tasks.resolution                buckets  \
0         12488335                     rules             Hair Color   
1         12527594                     rules             Hair Color   
2         12527608                     rules             Hair Color   
3         12527616                     rules             Hair Color   
4         12653573                     rules                Mascara   
...            ...                       ...                    ...   
402032    77031444                      bulk  Bookcases/Bookshelves   
402033    77360476                      bulk  Bookcases/Bookshelves   
402034    77526072                      bulk                 Chairs   
402035    77526085                      bulk                 Chairs   
402036    77535655                      bulk                 Chairs   

        curation_task_id  curated_by       date  
0              764545353         762 2021-05-28  
1              764545356         762 2021-05-28  
2              764545357         762 2021-05-28  
3              764545358         762 2021-05-28  
4              764545014         762 2021-05-28  
...                  ...         ...        ...  
402032         763989696         762 2021-05-28  
402033         763989723         762 2021-05-28  
402034         763989491         762 2021-05-28  
402035         763989608         762 2021-05-28  
402036         763989522         762 2021-05-28  

[402037 rows x 10 columns]

In [17]:
df[(df['date']>'04-30-2021')&(df['curated_by']==762)]

attributes    snake_case_name     value  customer_id  \
0       Application Style  application_style  Brush-On            5   
1       Application Style  application_style  Brush-On            5   
2       Application Style  application_style  Brush-On            5   
3       Application Style  application_style  Brush-On            5   
4       Application Style  application_style  Brush-On            5   
...                   ...                ...       ...          ...   
402032    Weight Capacity    weight_capacity       n/a            5   
402033    Weight Capacity    weight_capacity       n/a            5   
402034    Weight Capacity    weight_capacity       n/a            5   
402035    Weight Capacity    weight_capacity       n/a            5   
402036    Weight Capacity    weight_capacity       n/a            5   

       external_id curation_tasks.resolution                buckets  \
0         12488335                     rules             Hair Color   
1         12527594                     rules             Hair Color   
2         12527608                     rules             Hair Color   
3         12527616                     rules             Hair Color   
4         12653573                     rules                Mascara   
...            ...                       ...                    ...   
402032    77031444                      bulk  Bookcases/Bookshelves   
402033    77360476                      bulk  Bookcases/Bookshelves   
402034    77526072                      bulk                 Chairs   
402035    77526085                      bulk                 Chairs   
402036    77535655                      bulk                 Chairs   

        curation_task_id  curated_by       date  
0              764545353         762 2021-05-28  
1              764545356         762 2021-05-28  
2              764545357         762 2021-05-28  
3              764545358         762 2021-05-28  
4              764545014         762 2021-05-28  
...                  ...         ...        ...  
402032         763989696         762 2021-05-28  
402033         763989723         762 2021-05-28  
402034         763989491         762 2021-05-28  
402035         763989608         762 2021-05-28  
402036         763989522         762 2021-05-28  

[247292 rows x 10 columns]

In [33]:
df.groupby(['date']).count()

attributes  snake_case_name  external_id  customer_id  value  \
date                                                                       
2021-04-05           7                7            7            7      7   
2021-04-07        7886             7886         7886         7886   7886   
2021-04-09       20163            20163        20163        20163  20163   
2021-04-10        1496             1496         1496         1496   1496   
2021-04-13        5256             5256         5256         5256   5256   
2021-04-14        1343             1343         1343         1343   1343   
2021-04-15        4893             4893         4893         4893   4893   
2021-04-20       16897            16897        16897        16897  16897   
2021-04-21       69939            69939        69939        69939  69939   
2021-04-22          18               18           18           18     18   
2021-04-23       13825            13825        13825        13825  13825   
2021-04-26        6909             6909         6909         6909   6909   
2021-04-27        8791             8791         8791         8791   8791   
2021-05-02         907              907          907          907    907   
2021-05-03        1097             1097         1097         1097   1097   
2021-05-04        4773             4773         4773         4773   4773   
2021-05-05        6075             6075         6075         6075   6075   
2021-05-06       10303            10303        10303        10303  10303   
2021-05-07         332              332          332          332    332   
2021-05-11        7837             7837         7837         7837   7837   
2021-05-12       13545            13545        13545        13545  13545   
2021-05-13        7018             7018         7018         7018   7018   
2021-05-17         514              514          514          514    514   

            resolution  buckets  curated_by  
date                                         
2021-04-05           7        7           7  
2021-04-07        7886     7886        7886  
2021-04-09       20163    20163       20163  
2021-04-10        1496     1496        1496  
2021-04-13        5256     5256        5256  
2021-04-14        1343     1343        1343  
2021-04-15        4893     4893        4893  
2021-04-20       16897    16897       16897  
2021-04-21       69939    69939       69939  
2021-04-22          18       18          18  
2021-04-23       13825    13825       13825  
2021-04-26        6909     6909        6909  
2021-04-27        8791     8791        8791  
2021-05-02         907      907         907  
2021-05-03        1097     1097        1097  
2021-05-04        4773     4773        4773  
2021-05-05        6075     6075        6075  
2021-05-06       10303    10303       10303  
2021-05-07         332      332         332  
2021-05-11        7837     7837        7837  
2021-05-12       13545    13545       13545  
2021-05-13        7018     7018        7018  
2021-05-17         514      514         514

In [25]:
# parameters
customer_id = '134'
formatted_attribute = 'Size Depth Cookware'
attribute = formatted_attribute.lower().replace(' ','_').replace('-','_')

# buckets = """Window Blinds"""
buckets = """Multiple Buckets"""

regex_pattern=r'(?i)(x\s?\d+.?\d*\".?D)|(.?.?.?.?.?Depth.?.?.?.?)()'

value='%n/a%'
customer_name='%bedbathandbeyond%'

params = {'customer_id': customer_id ,
          'attribute': attribute,
          'buckets': str(buckets.split('\t'))[1:-1],
          'value':value,
          'customer_name':customer_name
         }
curation_col = f'Q:{attribute}'
curated_by=762

In [26]:
customer_id = '134'
motion = query_from_file(file_name='query/curated_by.sql', params=params)
motion['date']=pd.to_datetime(motion['date'])

In [28]:
motion[motion['date']>'04-13-2021']

attributes               snake_case_name external_id  \
0                  Outside Diameter              outside_diameter    00106572   
1                  Outside Diameter              outside_diameter    00106590   
2                  Outside Diameter              outside_diameter    00106607   
3                  Outside Diameter              outside_diameter    00228424   
4                  Outside Diameter              outside_diameter    00228428   
...                             ...                           ...         ...   
47368  V-Belt Outside Length Inches  v_belt_outside_length_inches    10544311   
47369  V-Belt Outside Length Inches  v_belt_outside_length_inches    10899153   
47370  V-Belt Outside Length Inches  v_belt_outside_length_inches    10899154   
47371  V-Belt Outside Length Inches  v_belt_outside_length_inches    10899155   
47372  V-Belt Outside Length Inches  v_belt_outside_length_inches    10899156   

       customer_id                   value resolution       buckets  \
0              134  2.5001 - 3.0000 Inches      rules  Idler Pulley   
1              134  2.5001 - 3.0000 Inches      rules  Idler Pulley   
2              134  1.5001 - 2.0000 Inches      rules  Idler Pulley   
3              134            0 - 5 Inches      rules   Belt Pulley   
4              134            0 - 5 Inches      rules   Belt Pulley   
...            ...                     ...        ...           ...   
47368          134          26 - 50 Inches      rules        V-Belt   
47369          134                     n/a       bulk        V-Belt   
47370          134                     n/a       bulk        V-Belt   
47371          134                     n/a       bulk        V-Belt   
47372          134                     n/a       bulk        V-Belt   

       curated_by       date  
0             762 2021-05-10  
1             762 2021-05-10  
2             762 2021-05-10  
3             762 2021-05-10  
4             762 2021-05-10  
...           ...        ...  
47368         762 2021-05-06  
47369         762 2021-05-06  
47370         762 2021-05-06  
47371         762 2021-05-06  
47372         762 2021-05-06  

[47373 rows x 9 columns]

In [10]:
motion.groupby(['customer_id']).count()

attributes  snake_case_name  external_id  value  resolution  \
customer_id                                                                
134               66402            66402        66402  66402       66402   

             buckets  curated_by   date  
customer_id                              
134            66402       66402  66402

In [ ]:
df['attributes'].explode().value_counts()

In [ ]:
def re_extract(pattern, txt):
    matches = re.findall(pattern, txt)
    tmp_matches = []
    for match in matches:
        for tup in match:
            if tup != '':
                tmp_matches.append(tup)
    return list(set(tmp_matches))

In [ ]:
df['matches'] = df['long_desc'].apply(lambda x: re_extract(regex_pattern, x))
df

In [ ]:
df[df['matches'].astype(str)!='[]']

In [ ]:
# df[df['external_id'].astype(str)=='69600196']

In [ ]:
df['matches'].explode().value_counts()

In [ ]:
def split_col(data):
    data['matches']=data['matches'].str.get(0)
#     data=data.dropna()
    data['matches']=data['matches'].apply(lambda x:x.lower())
#     data[['length', 'width']]=data['matches'].str.split(" x ",expand=True,)

In [ ]:
def drop_characters(data):
    data['length']=data['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['width']=data['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\s|\w*\.\w*\.|\-|\'",'',str(measure)))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('"', ''))
    data['width']=data['width'].apply(lambda quote: quote.replace('"', ''))
    data['length']=data['length'].replace("'", '').apply(lambda quote: quote.replace('-',''))
    data['width']=data['width'].apply(lambda quote: quote.replace('-',''))

In [ ]:
def replace_none(data):
    data['width']=data['width'].apply(lambda x: 0 if x == '' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == '' else x)
    data['width']=data['width'].apply(lambda x: 0 if x == 'None' else x)
    data['length']=data['length'].apply(lambda x: 0 if x == 'None' else x)

In [ ]:
def add_quotes(data):
    data['length']=data['length'].apply(lambda x: str(x) + "'")
    data['width']=data['width'].apply(lambda x: str(x) + "'")

In [ ]:
def min_max(data):
    data['length']=data['length'].astype(float)
    data['width']=data['width'].astype(float)
    data['Max'] = data[['length', 'width']].values.max(1)
    data['Min'] = data[['length', 'width']].values.min(1)

In [ ]:
def remove_fractions(numbers):
    if numbers.find('-') > -1:
        numbers = literal_eval(numbers)
        inte, fract = numbers.split(' ')
        nom, den = fract.split('/')
        tmp = float(nom)/float(den)
        num = int(inte) + tmp
        return str(num)
    else:
        return numbers

In [ ]:
split_col(df) #might throw an error and have to drop nane

In [ ]:
df['width']=df['width'].astype(str).apply(lambda x: 0 if x == 'nan' else x)
df

In [ ]:
df['length']=df['length'].astype(str)
df['width']=df['width'].astype(str)
df['length']=df['length'].apply(lambda quote: quote.replace('-', ' '))
df['width']=df['width'].apply(lambda quote: quote.replace('-', ' '))
df['length']=df['length'].apply(lambda quote: quote.replace('inch', ''))
df['width']=df['width'].apply(lambda quote: quote.replace('inch', ''))
df

In [ ]:
df['length']=df['length'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
df['width']=df['width'].apply(lambda measure: re.sub(r"[a-z]|[A-z]|\w*\.\w*\.|\-|\'",'',str(measure)))
df

In [ ]:
a=df['length'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['length']=c
df

In [ ]:
a=df['width'].to_list()
c=[]
for i in range(len(a)):
    b=float(sum(Fraction(s) for s in a[i].split()))
    c.append(b)
df['width']=c
df

In [ ]:
# x=df
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)
# x['new'] = c
# x

In [ ]:
# a=x['length'].to_list()
# c=[]
# for i in range(len(a)):
#     b=float(sum(Fraction(s) for s in a[i].split()))
#     c.append(b)

In [ ]:
# remove_fractions(df['length'])

In [ ]:
# x=df
# a=eval(x['length'][3:4].astype(str))
# # numbers = eval(a)

In [ ]:
# float(sum(Fraction(s) for s in a))

In [ ]:
# from fractions import Fraction
# float(sum(Fraction(s) for s in '  12 1/2'.split()))

In [ ]:
# x=df
# x['length'] = x['length'].apply(lambda quote: quote.replace('-', ''))
# x['length'] = x['length'].apply(lambda quote: quote.replace('inch', ''))
# # x['length'] = x['length'].apply(lambda x: x.str)
# x

In [ ]:
# x[['first', 'second']]=x['length'].str.split("/",expand=True,)
# x[['third', 'fourth']= x['first'].split()
# result = x['third'] + (float(x['first'])/float(x['second']))
# result

In [ ]:
# num,den = s.split( '/' )
# wh, num = num.split()
# result = wh + (float(num)/float(den))

In [ ]:
# a=x['length'][3:4]
# a.str.split(' ')
# b= [[" ".join(tag.split(" ")[:2]) , " ".join(tag.split(" ")[2:])] for tag in a][0]
# b

In [ ]:
# c=b[1]
# c.split('/')

In [ ]:
# b[0]
# b

In [ ]:
# a.apply(lambda x: sum(Fraction(x.split('/'))))

In [ ]:
# float(sum(Fraction(s) for s in x['length'].split()))
# x['length']=x['length'].apply(lambda a: float(sum(Fraction(a.split()))))

In [ ]:
# for i in x['length']:
#     if x['length'][i].find('/') > -1:
#         float(sum(Fraction(s) for s in x['length'][i].split()))
#     else:
#         x['length']

In [ ]:
# float(sum(Fraction(s) for s in x['length'][i].split()))
# .apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))

In [ ]:
# def fraction_combo(numbers):
#     if numbers.find('/') > -1:
#         numbers.apply(lambda x: float(sum(Fraction((s) for s in numbers.split()))))
#         #numbers = literal_eval(numbers)
#         #inte, fract = numbers.split(' ')
#         #nom, den = fract.split('/')
#         #tmp = float(nom)/float(den)
#         #num = int(inte) + tmp
#         return str(numbers)
#     else:
#         return numbers

In [ ]:
# a=x['length']

In [ ]:
# remove_fractions(df)

In [ ]:
# x['length'] = x['length'].apply(lambda x: remove_fractions(x))
# x.head

In [ ]:
# x['length'] = x['length'].apply(lambda x: x.split(' '))
# x.head()

In [ ]:
min_max(df) #error base 10 when there is only a value for max and not min

In [ ]:
def na_to_gsheet(df):
    url = 'https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0'
    wks = 'n/a'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    gc = gcloud.Gsheets()
    gc.save_to_gsheet(url, wks, non_matches)

In [ ]:
na_to_gsheet(df)

In [ ]:
# # range function builder
# string = range_string.replace(metric, '')
# lst = string.split('\n')
# builder_base = f'''
# def range_app(num):
# '''
# for val in lst:
#     if val.find(' - ') >-1:
#         original_val = val
#         lower, upper = val.split(' - ')
#         inner_f = f'''
#     if num >= {lower} and num <= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
#     if val.find('Greater than ') >-1:
#         original_val = val
#         upper = val.replace('Greater than ','')
#         inner_f = f'''
#     if num >= {upper}:
#         return "{original_val}{metric}"
#         '''
#         builder_base = builder_base + inner_f
# final_else = '''
#     else:
#         return r"n/a"
# '''
# exec(builder_base + final_else)
# print( builder_base + final_else)

In [ ]:
df

In [ ]:
matches=df
matches['Max']=matches['Max'].astype(str).apply(lambda quote: quote.replace('.0', ''))
matches

In [ ]:
matches[curation_col] = matches['Max'].astype(str)+' in'
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
matches

In [ ]:
## KWP
# matches[curation_col]=matches['matches']
# matches['external_id']=matches['external_id'].apply('="{}"'.format)
# matches

In [ ]:
error----Switch Max and Min when doing length and width

In [ ]:
gc = gcloud.Gsheets()

In [ ]:
gc.save_to_gsheet('https://docs.google.com/spreadsheets/d/1A3dXssoEuvgun6YaO5jxM8P6_zZQ9KByEIZlhovF2gU/edit#gid=0','ul',matches)

# send to the folder for upload

In [ ]:
def looks_good(customer, attribute, df, matches):
    drive_path = f'G:/Shared drives/GroupBy Public/Customer Success/.Enrich/Platform Upload Trail/{customer}/_Ready For Upload/Shawn'
    non_matches = df[df['matches'].astype(str) == '[]']
    non_matches[curation_col] = r'n/a'
    non_matches.to_csv(f'{drive_path}/BBBY - {attribute} - na upload {buckets}.csv',index=False)
    matches.to_csv(f'{drive_path}/BBBY - {attribute} - match upload {buckets}.csv',index=False)

In [ ]:
looks_good('Bed Bath & Beyond', attribute, df, matches)